<a href="https://colab.research.google.com/github/TheJojoJoseph/Assignment3SNA/blob/main/Assignment3SNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. **1. Loading Datasets: **The gml_to_dataframe function is used to load the datasets (dolphins.gml, karate.gml, and football.gml). This method reads GML files and converts them to Pandas DataFrames that represent the graph's edges.
2. **2. Data Cleaning: **To standardise the dataset, the function clean_dataset_fixed extracts only the first two columns (node1, node2) that indicate the edges and verifies that the data is of the integer type.
convert_cleaned_to_adjacency_matrix: This function creates adjacency matrices from cleaned datasets. When an edge connects two nodes, this matrix will have 1; otherwise, it will have 0.
3. **infoNode algorithm,** calculate_internal_force, determines the internal force between two nodes by utilising their shared neighbours and Jaccard coefficient.
infonode with internal force: This program uses the InfoNode algorithm to find communities.
locates high degree centrality seed nodes.
By choosing the top-k neighbours with the highest internal force, communities are expanded.
Communities that have a lot in common are combined to prevent over-segmentation.
4. **Girvan-Newman community detection technique** is implemented by girvan_newman_algo, which repeatedly eliminates edges with the highest betweenness centrality until communities are formed.
5. **Flattening Communities:**
flatten_communities_safe: Flattens community structures into a list where each node is labeled with its community ID, ensuring no out-of-range errors.
6. Network Statistics:
compute_network_stats: Computes basic statistics for each graph (number of nodes, edges, average degree).
network_stats_from_adj: Helper function that applies the above statistics calculation to an adjacency matrix.
7. **Community Detection on Three Datasets:**
Runs both the InfoNode and Girvan-Newman algorithms on the dolphins, karate, and football datasets, producing communities for each.
8. **NMI Calculation:**
NMI (Normalized Mutual Information) is used to compare the similarity between the community structures produced by the two algorithms.
For each dataset (Dolphins, Karate, Football):
Communities from InfoNode and Girvan-Newman are flattened into label vectors.
The NMI between the two community label vectors is computed to quantify the similarity between the detected communities.

In [18]:
#3 datasets (Dolphins, Karate, Football) -  Added in GITfile and Zip

# Loading
from google.colab import files
# uploaded = files.upload() # I have already uploaded so commented.

import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.community import girvan_newman
from sklearn.metrics import normalized_mutual_info_score as nmi


def gml_to_dataframe(file_path):  #<--Load the GML file and convert to a DataFrame
    G = nx.read_gml(file_path, label=None)  # Use label=None to avoid label errors
    edges = [(u, v) for u, v in G.edges()]
    return pd.DataFrame(edges, columns=['node1', 'node2'])

dolphins_df = gml_to_dataframe('dolphins.gml')
karate_df = gml_to_dataframe('karate.gml')
football_df = gml_to_dataframe('football.gml')

print('Dolphins_DataSet',dolphins_df.head())
print('Karate_DataSet',karate_df.head())
print('Football_DataSet',football_df.head())


# Load datasets - If CSV
# dolphins_df = pd.read_csv('dolphins.csv', header=None)
# karate_df = pd.read_csv('karate.csv', header=None)
# football_df = pd.read_csv('football.csv', header=None)

def clean_dataset_fixed(data, has_weights=False): # Function to clean and convert the dataset into adjacency matrix
    cleaned_data = data.copy().reset_index(drop=True)
    if cleaned_data.dtypes.iloc[0] == 'object':
        cleaned_data = cleaned_data[cleaned_data.columns[0]].str.strip().str.split(expand=True)
    cleaned_data = cleaned_data.iloc[:, :2]
    cleaned_data.columns = ['node1', 'node2']
    return cleaned_data.astype(int)

def convert_cleaned_to_adjacency_matrix(cleaned_data):  # -->Convert cleaned datasets to adjacency matrices
    """Converts cleaned dataset into an adjacency matrix."""
    max_node = max(cleaned_data['node1'].max(), cleaned_data['node2'].max())
    adj_matrix = pd.DataFrame(0, index=range(max_node+1), columns=range(max_node+1))
    for _, row in cleaned_data.iterrows():
        adj_matrix.iloc[row['node1'], row['node2']] = 1
        adj_matrix.iloc[row['node2'], row['node1']] = 1  # Symmetric
    return adj_matrix

# Clean the datasets && adjacency matrices
dolphins_cleaned = clean_dataset_fixed(dolphins_df)
karate_cleaned = clean_dataset_fixed(karate_df)
football_cleaned = clean_dataset_fixed(football_df)

dolphins_adj_matrix = convert_cleaned_to_adjacency_matrix(dolphins_cleaned)
karate_adj_matrix = convert_cleaned_to_adjacency_matrix(karate_cleaned)
football_adj_matrix = convert_cleaned_to_adjacency_matrix(football_cleaned)

# InfoNode Algorithm
def calculate_internal_force(G, node1, node2):
    degree_product = G.degree(node1) * G.degree(node2)
    neighbors1 = set(G.neighbors(node1))
    neighbors2 = set(G.neighbors(node2))
    intersection = len(neighbors1 & neighbors2)
    union = len(neighbors1 | neighbors2)
    jaccard_coefficient = intersection / union if union > 0 else 0
    return degree_product * jaccard_coefficient

def infonode_with_internal_force(adj_matrix, k=5):
    G = nx.from_pandas_adjacency(adj_matrix)
    degree_centrality = nx.degree_centrality(G)
    avg_centrality = np.mean(list(degree_centrality.values()))
    seeds = [node for node, centrality in degree_centrality.items() if centrality > avg_centrality]
    communities = {seed: [seed] for seed in seeds}
    visited = set(seeds)
    for seed in seeds:
        neighbors = list(set(G.neighbors(seed)) - visited)
        if not neighbors:
            continue
        internal_forces = [(neighbor, calculate_internal_force(G, seed, neighbor)) for neighbor in neighbors]
        internal_forces = sorted(internal_forces, key=lambda x: -x[1])  # Sort by descending internal force
        top_k_neighbors = [n for n, _ in internal_forces[:k]]
        communities[seed].extend(top_k_neighbors)
        visited.update(top_k_neighbors)
    community_list = list(communities.values()) #Merge communities if they overlap significantly to reduce over-segmentation
    merged_communities = []
    while community_list:
        current_community = set(community_list.pop(0))
        merged = False
        for i, other_community in enumerate(community_list):
            if len(current_community & set(other_community)) > 0:
                current_community.update(other_community)
                community_list.pop(i)
                merged = True
                break
        merged_communities.append(list(current_community))
    return merged_communities

# Girvan-Newman Algorithm
def girvan_newman_algo(adj_matrix):
    G = nx.from_pandas_adjacency(adj_matrix)
    comp = girvan_newman(G)
    communities = tuple(sorted(c) for c in next(comp))
    return communities

#  flatten communities helper for NMI comparison
def flatten_communities(communities, num_nodes):
    labels = [-1] * num_nodes
    for comm_id, nodes in communities.items():
        for node in nodes:
            labels[node] = comm_id
    return labels

def flatten_communities_list(communities, num_nodes):
    labels = [-1] * num_nodes
    for comm_id, nodes in enumerate(communities):
        for node in nodes:
            labels[node] = comm_id
    return labels

def compute_network_stats(G): #basic network statistics
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    return num_nodes, num_edges, avg_degree

#Iterator
def network_stats_from_adj(adj_matrix):
    G = nx.from_pandas_adjacency(adj_matrix)
    return compute_network_stats(G)

# Flatten communities for NMI comparison
def flatten_communities_safe(communities, num_nodes):
    """Flattens community structure into a list where index is the node and value is the community label, ensuring correct indexing."""
    labels = [-1] * (num_nodes + 1)  # Ensure size accommodates for all nodes, even if non-continuous
    if isinstance(communities, dict):
        for comm_id, nodes in communities.items():
            for node in nodes:
                if node < len(labels):
                    labels[node] = comm_id
    else:
        for comm_id, nodes in enumerate(communities):
            for node in nodes:
                if node < len(labels):
                    labels[node] = comm_id
    return labels[:num_nodes]

# --------------- Apply the algorithms and calculate NMI for Dolphins, Karate, and Football ---------------------------------
# InfoNode for Dolphins
infonode_communities_dolphins = infonode_with_internal_force(dolphins_adj_matrix)
gn_communities_dolphins = girvan_newman_algo(dolphins_adj_matrix)

# InfoNode for Karate
infonode_communities_karate = infonode_with_internal_force(karate_adj_matrix)
gn_communities_karate = girvan_newman_algo(karate_adj_matrix)

# InfoNode for Football
infonode_communities_football = infonode_with_internal_force(football_adj_matrix)
gn_communities_football = girvan_newman_algo(football_adj_matrix)

# Network stats for Dolphins
dolphins_stats = network_stats_from_adj(dolphins_adj_matrix)
dolphins_info_communities = len(infonode_communities_dolphins)
dolphins_gn_communities = len(gn_communities_dolphins)

# Network stats for Karate
karate_stats = network_stats_from_adj(karate_adj_matrix)
karate_info_communities = len(infonode_communities_karate)
karate_gn_communities = len(gn_communities_karate)

# Network stats for Football
football_stats = network_stats_from_adj(football_adj_matrix)
football_info_communities = len(infonode_communities_football)
football_gn_communities = len(gn_communities_football)

# Flatten communities for NMI comparison
num_nodes_dolphins_result = dolphins_adj_matrix.shape[0]
infonode_labels_dolphins = flatten_communities_safe(infonode_communities_dolphins, num_nodes_dolphins_result)
gn_labels_dolphins = flatten_communities_safe(gn_communities_dolphins, num_nodes_dolphins_result)
nmi_dolphins = nmi(infonode_labels_dolphins, gn_labels_dolphins)

num_nodes_karate_result = karate_adj_matrix.shape[0]
infonode_labels_karate = flatten_communities_safe(infonode_communities_karate, num_nodes_karate_result)
gn_labels_karate = flatten_communities_safe(gn_communities_karate, num_nodes_karate_result)
nmi_karate = nmi(infonode_labels_karate, gn_labels_karate)

num_nodes_football_result = football_adj_matrix.shape[0]
infonode_labels_football = flatten_communities_safe(infonode_communities_football, num_nodes_football_result)
gn_labels_football = flatten_communities_safe(gn_communities_football, num_nodes_football_result)
nmi_football = nmi(infonode_labels_football, gn_labels_football)

def print_nmi(dataset_name, results):
    print(f"{dataset_name} Dataset: NMI = {results}")

# --------------------   NMI results for all three datasets. ----------------------
print_nmi("Dolphins", round(nmi_dolphins, 4))
print_nmi("Karate", round(nmi_karate, 4))
print_nmi("Football", round(nmi_football, 4))



Dolphins_DataSet    node1  node2
0      0     10
1      0     14
2      0     15
3      0     40
4      0     42
Karate_DataSet    node1  node2
0      1      2
1      1      3
2      1      4
3      1      5
4      1      6
Football_DataSet    node1  node2
0      0      1
1      0      4
2      0      9
3      0     16
4      0     23
Dolphins Dataset: NMI = 0.2681
Karate Dataset: NMI = 0.4407
Football Dataset: NMI = 0.2253
